In [17]:
import h5py
import numpy as np
import scipy.io
import cv2
import os
from matplotlib import pyplot as plt

<h2>PaviaUni</h2>

In [2]:
paviaU = scipy.io.loadmat('dataset/Pavia_uni/PaviaU.mat')
npPaviaU=np.array(paviaU['paviaU'])

print ("Height:\t\t%i pixels\nWidth:\t\t%i pixels\nChannels:\t%i" % npPaviaU.shape)
print ("data-type: %s " % npPaviaU.dtype)

Height:		610 pixels
Width:		340 pixels
Channels:	103
data-type: uint16 


<h2>PaviaUni Groundtruth</h2>

In [3]:
paviaU_gt = scipy.io.loadmat('dataset/Pavia_uni/PaviaU_gt.mat')

npPaviaU_gt=np.array(paviaU_gt['paviaU_gt'])
npPaviaU_gt

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [2, 2, 2, ..., 0, 0, 0],
       [2, 2, 2, ..., 0, 0, 0],
       [2, 2, 2, ..., 0, 0, 0]], dtype=uint8)

<b>PaviaUni Groundtruth group by class</b>

In [4]:
w_dim = 610
h_dim = 340

D = { '0': 0, '1-Asphalt': 0, '2-Meadows': 0, '3-Gravel': 0, '4-Trees': 0, '5-Painted_metal_sheets': 0, '6-Bare_Soil': 0, '7-Bitumen': 0, '8-Self-Blocking_Bricks': 0, '9-Shadows': 0 }
for i in range(w_dim):
    for j in range(h_dim):
        if(npPaviaU_gt[i][j] == 0):
            D['0']+=1
        elif(npPaviaU_gt[i][j] == 1):
            D['1-Asphalt']+=1
        elif(npPaviaU_gt[i][j] == 2):
            D['2-Meadows']+=1
        elif(npPaviaU_gt[i][j] == 3):
            D['3-Gravel']+=1      
        elif(npPaviaU_gt[i][j] == 4):
            D['4-Trees']+=1
        elif(npPaviaU_gt[i][j] == 5):
            D['5-Painted_metal_sheets']+=1
            #print("5:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 6):
            D['6-Bare_Soil']+=1
            #print("6:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 7):
            D['7-Bitumen']+=1
            #print("7:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 8):
            D['8-Self-Blocking_Bricks']+=1
            #print("8:{x},{y}".format(x=i,y=j))  
        elif(npPaviaU_gt[i][j] == 9):
            D['9-Shadows']+=1            
            #print("9:{x},{y}".format(x=i,y=j))  
            
D

{'0': 164624,
 '1-Asphalt': 6631,
 '2-Meadows': 18649,
 '3-Gravel': 2099,
 '4-Trees': 3064,
 '5-Painted_metal_sheets': 1345,
 '6-Bare_Soil': 5029,
 '7-Bitumen': 1330,
 '8-Self-Blocking_Bricks': 3682,
 '9-Shadows': 947}

In [53]:
6631+18649+2099+3064+1345+5029+1330+3682+947

42776

<h2>Create h5</h2>

In [5]:
def labeled(r,g,b):
    label = -1
    if(r==192 and g==192 and b==192):
        label = 0
    elif(r==0 and g==255 and b==0):
        label = 1   
    elif(r==255 and g==255 and b==0):
        label = 2  
    elif(r==0 and g==128 and b==0):
        label = 3  
    elif(r==255 and g==0 and b==255):
        label = 4  
    elif(r==41 and g==82 and b==165):
        label = 5  
    elif(r==128 and g==0 and b==128):
        label = 6  
    elif(r==0 and g==0 and b==255):
        label = 7        
    elif(r==0 and g==255 and b==255):
        label = 8  
        
    return label        

<h3>Create Train HDF5</h3>

In [100]:
#TRAIN
w_dim = 340
h_dim = 610
z_dim = 103

#Train Image
paviaU_trainImage = cv2.imread('images/pavia_university/Train_University.bmp')

textfile_train = open("/home/yavuz/caffe/data/paviaUni_new/train.txt", "w")

ii=0
for i in range(h_dim):
    for j in range(w_dim):
        
        r = paviaU_trainImage[i][j][0]
        g = paviaU_trainImage[i][j][1]
        b = paviaU_trainImage[i][j][2]
        if(r!=0 or g!=0 or b!=0):
            #print("{x},{y}".format(x=i,y=j))
            
            lbl = labeled(r,g,b) #npPaviaU_gt[i][j]
            
            h5_Train = h5py.File("/home/yavuz/caffe/data/paviaUni_new/train/train_{x}_{y}_{label}.hdf5".format(x=i,y=j,label=lbl), 'w')
            h5_Train.create_dataset('data', (1, 1, 103), dtype='int64')
            h5_Train.create_dataset('label', (1,1), dtype='int8')
            
            a = np.empty((1,1, 103))
            for z in range(z_dim):
                a[0][0][z] = npPaviaU[i][j][z]
                
            h5_Train['data'][0] = a
            h5_Train['label'][0] = lbl
            
            h5_Train.close()
            
            

            textfile_train.write("/home/yavuz/caffe/data/paviaUni_new/train/train_{x}_{y}_{label}.hdf5\n".format(x=i,y=j,label=lbl))
            
            ii+=1
   
            
textfile_train.close()          
print("DONE! - {i}".format(i=ii))            

DONE! - 3921


In [105]:
test = h5py.File("/home/yavuz/caffe/data/paviaUni_new/train/train_455_10_2.hdf5",'r')
test['data']

<HDF5 dataset "data": shape (1, 1, 103), type "<i8">

In [106]:
test['data'][0]

array([[ 767,  854, 1091, 1097, 1122, 1156, 1100, 1071, 1059, 1102, 1062,
        1006, 1000, 1005, 1030, 1056, 1096, 1086, 1099, 1134, 1150, 1146,
        1155, 1166, 1192, 1243, 1255, 1249, 1274, 1301, 1321, 1328, 1354,
        1386, 1387, 1397, 1399, 1399, 1409, 1409, 1424, 1445, 1455, 1455,
        1452, 1452, 1442, 1441, 1449, 1459, 1496, 1503, 1471, 1454, 1434,
        1431, 1448, 1486, 1495, 1475, 1475, 1476, 1478, 1478, 1462, 1460,
        1481, 1497, 1522, 1523, 1507, 1488, 1487, 1483, 1497, 1533, 1552,
        1561, 1555, 1562, 1543, 1491, 1436, 1462, 1562, 1584, 1537, 1489,
        1481, 1498, 1535, 1545, 1530, 1498, 1478, 1463, 1467, 1495, 1512,
        1524, 1496, 1491, 1463]])

<h3>Create Test HDF5</h3>

In [101]:
#TEST
w_dim = 340
h_dim = 610
z_dim = 103

#TEST Image
paviaU_testImage = cv2.imread('images/pavia_university/Test_University.bmp')

textfile_test = open("/home/yavuz/caffe/data/paviaUni_new/test.txt", "w")

ii=0
for i in range(h_dim):
    for j in range(w_dim):
        
        r = paviaU_testImage[i][j][0]
        g = paviaU_testImage[i][j][1]
        b = paviaU_testImage[i][j][2]
        if(r!=0 or g!=0 or b!=0):
            #print("{x},{y}".format(x=i,y=j))
            
            lbl = labeled(r,g,b)
            
            h5_Train = h5py.File("/home/yavuz/caffe/data/paviaUni_new/test/test_{x}_{y}_{label}.hdf5".format(x=i,y=j,label=lbl), 'w')
            h5_Train.create_dataset('data', (1, 1, 103), dtype='int64')
            h5_Train.create_dataset('label', (1,1), dtype='int8')
            
            a = np.empty((1,1, 103))
            for z in range(z_dim):
                a[0][0][z] = npPaviaU[i][j][z]
                
            h5_Train['data'][0] = a
            h5_Train['label'][0] = lbl
            
            h5_Train.close()
            
            

            textfile_test.write("/home/yavuz/caffe/data/paviaUni_new/test/test_{x}_{y}_{label}.hdf5\n".format(x=i,y=j,label=lbl))
            
            ii+=1
   
            
textfile_test.close()          
print("DONE! - {i}".format(i=ii))     

DONE! - 42776
